In [5]:
from pynq import Overlay
from pynq import MMIO

import math


ol = Overlay("./cordic_wrapper.bit") # designate a bitstream to be flashed to the FPGA
ol.download() # flash the FPGA

mul_ip = MMIO(0x40000000, 0x10000) # (IP_BASE_ADDRESS, ADDRESS_RANGE), told to us in Vivado

In [6]:
def float_to_fixed_point_int(value, total_bits=16, integer_bits=3):
    """
    Convert a floating-point number to a fixed-point integer.
    Uses two's complement for negative values.
    """
    fractional_bits = total_bits - integer_bits
    scaled_value = int(round(value * (1 << fractional_bits)))

    if scaled_value < 0:
        scaled_value = (1 << total_bits) + scaled_value

    return scaled_value & ((1 << total_bits) - 1)


def fixed_point_int_to_float(fixed_val, total_bits=16, integer_bits=3):
    """
    Convert a fixed-point integer back to a floating-point number.
    Assumes two's complement representation.
    """
    fractional_bits = total_bits - integer_bits

    if fixed_val >= (1 << (total_bits - 1)):
        fixed_val -= (1 << total_bits)

    return fixed_val / (1 << fractional_bits)

In [7]:
def cart2pol(x, y):
    if x > 1 or x < -1 or y > 1 or y < -1:
        print("Wrong input for x or y")
    
    fp_x = float_to_fixed_point_int(x)
    fp_y = float_to_fixed_point_int(y)
    
    mul_ip.write(0x10, fp_x)
    mul_ip.write(0x18, fp_y)
    print("input x:", mul_ip.read(0x10)) # confirm that our value was written correctly to the fabric
    print("input y:", mul_ip.read(0x18)) # confirm that our value was written correctly to the fabric
    
    mul_ip.write(0x00, 1) # set ap_start to 1 which initiates the process we wrote to the fabric
    
    fp_r = mul_ip.read(0x20)
    fp_theta = mul_ip.read(0x30) 
    
    r     = fixed_point_int_to_float(fp_r)
    theta = fixed_point_int_to_float(fp_theta)
        
    golden_r = math.sqrt(x**2 + y**2)
    golden_theta = math.atan2(y, x)  # Returns angle in radians
    
    print("golden_r     = {},\t calculated r     = {}".format(golden_r, r))
    print("golden_theta = {},\t calculated theta = {}".format(golden_theta, theta))

    



In [8]:
cart2pol(0.8147, 0.1269)
cart2pol(0.6323, -0.2785)
cart2pol(-0.5469, -0.9575)
cart2pol(-0.4854, 0.7003)

input x: 6674
input y: 1040
golden_r     = 0.8245239232429827,	 calculated r     = 0.824951171875
golden_theta = 0.15452117380563432,	 calculated theta = 0.1536865234375
input x: 5180
input y: 63255
golden_r     = 0.6909164493627287,	 calculated r     = 0.69140625
golden_theta = -0.4148884123756094,	 calculated theta = -0.415771484375
input x: 61056
input y: 57692
golden_r     = 1.1026812141321716,	 calculated r     = 1.1031494140625
golden_theta = -2.089751217098013,	 calculated theta = -2.089599609375
input x: 61560
input y: 5737
golden_r     = 0.8520758475628798,	 calculated r     = 0.8526611328125
golden_theta = 2.176897679895153,	 calculated theta = 2.1763916015625
